Purpose:
Suggest best-fit teams and solo performers based on:
- Skill Match Score (from earlier module)
- Synergy Score (from the last module)
We’ll rank combinations and recommend:
- Top 3-person teams
- Top solo performers for tasks needing individual focus


In [6]:
import pandas as pd
from itertools import combinations

# Load skill match and synergy scores
df_skill = pd.read_csv("skill_match_scores.csv")
df_synergy = pd.read_csv("employee_synergy_scores.csv")

# Clean column names if needed
df_skill.columns = [col.strip() for col in df_skill.columns]
df_synergy.columns = [col.strip() for col in df_synergy.columns]

# Get unique employees
employees = df_skill["Employee Name"].unique()
team_combos = list(combinations(employees, 3))

# Function to get synergy score between two employees
def get_synergy(emp1, emp2):
    match = df_synergy[
        ((df_synergy["Employee 1"] == emp1) & (df_synergy["Employee 2"] == emp2)) |
        ((df_synergy["Employee 1"] == emp2) & (df_synergy["Employee 2"] == emp1))
    ]
    return match["Synergy Score"].values[0] if not match.empty else 0

# Build team scores
team_scores = []

for team in team_combos:
    # Filter skill scores for this team
    skill_total = df_skill[df_skill["Employee Name"].isin(team)]["Match Score"].sum()

    # Calculate synergy between all pairs
    synergy_total = (
        get_synergy(team[0], team[1]) +
        get_synergy(team[0], team[2]) +
        get_synergy(team[1], team[2])
    )

    team_scores.append({
        "Team Members": ", ".join(team),
        "Skill Score": skill_total,
        "Synergy Score": synergy_total,
        "Total Score": skill_total + synergy_total
    })

# Recommend top solo performers
solo_scores = df_skill.sort_values(by="Match Score", ascending=False)[["Employee Name", "Match Score"]]

# Save results
df_teams = pd.DataFrame(team_scores).sort_values(by="Total Score", ascending=False)
df_teams.to_csv("top_team_recommendations.csv", index=False)
solo_scores.to_csv("top_solo_recommendations.csv", index=False)

print("✅ Team and solo recommendations saved.")

✅ Team and solo recommendations saved.
